In [ ]:
#check if we can change shit.
# address issue here:
# https://www.kaggle.com/product-feedback/325765#2073421
# after done, click "restart session" from run tab.
!pip uninstall pyarrow -y
!pip install pyarrow==10.0.1
# !conda install -c conda-forge pyarrow -y
# remove default pyarrow!
mpath = "/opt/conda/lib/python3.7/site-packages/"
# under "pyarrow"
import os
dirs = os.listdir(mpath)
for d in dirs:
    if "pyarrow" in d:
        print(d)

In [ ]:
import pyarrow
# pyarrow.__version__ = '10.0.1'
print("version?",pyarrow.__version__)
print("file?", pyarrow.__file__)
# let's hack.


In [ ]:

# !pip install pyarrow==10.0.1
# https://www.kaggle.com/product-feedback/325765
# 
# !conda update pyarrow -y
import pyarrow
print("pyarrow version:", pyarrow.__version__)
!pip show pyarrow
import pyarrow
import os

print(f"\npath: {os.path.abspath(pyarrow.__file__)}\nversion: {pyarrow.__version__}")

In [ ]:
!pip install pfrl@git+https://github.com/voidful/pfrl.git
!pip install textrl==0.1.6
!pip install datasets
# you don't fucking upgrade!
!pip install gpuinfo
!pip install -U sentence-transformers
!pip install setfit

In [ ]:

from setfit import SetFitModel
# train new model later.
model_id = "lewtun/my-awesome-setfit-model"
model = SetFitModel.from_pretrained(model_id)
# model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

In [ ]:
def getVRAMUsage():
    from gpuinfo import GPUInfo
    memusage = GPUInfo.gpu_usage()
    mdict = dict(zip(*memusage))
    # this key is unstable. it cannot always get the first GPU card.
    print("dict:",mdict)
    data = mdict[list(mdict.keys())[0]]
    print("VRAM?",data,type(data)) # 1549. around.
    return data

getVRAMUsage()

In [ ]:
getVRAMUsage()

In [ ]:
# test how to use this setfit model
setfit_model = model
# this is just sentence transformer. does it works with long query?
# i wonder what "prompt-free" looks like.
setfit_test_data = ["i loved the spiderman movie!", "pineapple on pizza is the worst 🤮"]
val =  setfit_model(setfit_test_data)
print(val)
# what about certainties?
# this is binary. not so good for reward or adjustments.

In [ ]:
print(dir(model))

In [ ]:
val = model.predict_proba(setfit_test_data)
# now you have it.
# does it fucking works?
print(val,type(val),val.shape)

In [ ]:
# what is the actual probability, when compared as a whole?
import numpy
def getPositiveProbability(msent_in_array): # multiple elements maybe?
    val = setfit_model.predict_proba(msent_in_array)
    # let's not do this shall we?
    # whatever.
#     quad = val**2
    quad = val
    msum = numpy.sum(quad,axis=1)
    mval = quad[:,0] # what to do? elon musk speaks evil
    result = mval/msum
    return result
    # but this is something else.

val = getPositiveProbability(setfit_test_data)
print(val)

In [ ]:
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import logging
import sys
import pfrl
import torch
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')
# reference: https://github.com/voidful/fastpages/blob/master/_notebooks/2022-12-10-textrl-elon-musk.ipynb

In [ ]:
# on kaggle this is not

# you should mod this.
# local_files_only=True -> local_files_first
# try to modify it somehow?
# you would change the model
# modelId = "huggingtweets/elonmusk"
modelId = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(modelId)
model = AutoModelWithLMHead.from_pretrained(modelId)
model.eval()
model.cuda()
print("model loaded. please check VRAM.")
getVRAMUsage()
# why don't you print it?

In [ ]:
## warning! vram size is too big. close to 1.5GB even if not training.
## you'd better do this experiment on kaggle. 
## yes we better shift. change the code if needed. monitor ram usage.

# but what's the capacity? 16GB max for P100 kaggle.

# sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)
# # shit! what to do? revert?
# # OOM.
# # let's revert. fuck.
# getVRAMUsage()

In [ ]:
# get python env version?
# see if we can use walrus here.
import sys
print(sys.version) # shit. 3.7 < 3.8

In [ ]:
class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
      reward = 0
      if finish or len(predicted_list) >= self.env_max_length:
        if 1 < len(predicted_list):
          print('finished?',finish)
          print('response length?', len(predicted_list)) # counted for token.
          predicted_text = tokenizer.convert_tokens_to_string(predicted_list)
          # sentiment classifier
          # reward += sentiment(input_item[0]+predicted_text)[0][0]['score']
          # compute random reward. let's compute according to the letter 's'
          # count how many letter 's' inside the response.
          minput = input_item[0]
          print("input?",minput)
          mpredicted = predicted_text
          print('predicted?',mpredicted)
          total = minput+mpredicted
#           score = total.lower().count("s") / len(total)
          # now let's fix.
          score = getPositiveProbability([total])[0]
          score = float(score)
          print("score?",score)
          reward += score
      return reward

print("env prepared?")

In [ ]:
# different from original observation!
# i like typo.
observaton_list = [['i think dogecoin is']]
# observaton_list = [['i think dogecoin is', 'but it is not just about the dogecoin,']] # what is this format? dialog?
# obviously not markov.
# you cannot know what the model says next. you will only know if it responds and then you may reply.
# this behavior is online-learning, not in the damn dataset.

In [ ]:

env = MyRLEnv(model, tokenizer, observation_input=observaton_list)
actor = TextRLActor(env,model,tokenizer)
agent = actor.agent_ppo(update_interval=10, minibatch_size=10, epochs=10)
print('agent loaded')
getVRAMUsage() # 1493. economic?

In [ ]:
myInput = observaton_list[0]
print("input?", myInput)
ans = actor.predict(myInput)
# is it a training batch? what about the model temperature?
print("ans?", ans)
getVRAMUsage()

In [ ]:
print("about to train. please check VRAM.")
# how is the input consumed, by passing two string elements as a single list to `actor.predict`?
# shit now let's train!
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=300,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100, 
    save_best_so_far_agent=True,
    checkpoint_freq = 10*10000000000, # almost never save checkpoints.
    eval_interval=10,
    outdir='elon_musk_dogecoin', 
) # how to preserve best only?
getVRAMUsage()
# after training?

In [ ]:
# little brain. no valuable response.

# not too much! now let's check disk usage.
# ok now it is good. how comes?
agent.load("./elon_musk_dogecoin/best")
# check vram.
getVRAMUsage()
val = actor.predict(observaton_list[0])
print("VAL?",val)
# so it does include more letter 's' while maintains readability?

In [ ]:
# we try to replace the thing with more advanced shits. we score this thing.
# score labeled as such:
# same category, with qualified response.
# sure?

# also you might want to collect data then train.
# making it into ever-evolving shit.
# unsupervised -> supervised -> RL
# a whole loop.

# how to make it granualized?
